# Assignment 4 Group 16

## Import the necessary libraries

In [45]:
import pandas as pd
import numpy as np
from scipy.stats import t
from scipy.stats import norm
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [24]:
# import our custom functions
from Assignment4_lib import HSMeasurements, bootstrapStatistical, WHSMeasurements, \
    PrincCompAnalysis, plausibilityCheck

In [25]:
# fix the random seed
np.random.seed(42)

## Load the data from the files

In [26]:
# load the indexes as dictionary of {ticker: name}
with open('data/_indexes.csv', 'r') as f:
    # skip the first line
    indexes = {
        line.split(',')[1]: line.split(',')[2].strip()
        for line in f.readlines()[1:]
    }

In [27]:
# load the actual dataset as a dataframe
EuroStoxx50 = pd.read_csv('data/EUROSTOXX50_Dataset.csv', sep=',')
EuroStoxx50 = EuroStoxx50.set_index(pd.DatetimeIndex(EuroStoxx50['Date']))
EuroStoxx50 = EuroStoxx50.drop('Date', axis=1)
EuroStoxx50.head()

,ABI.BR,AD.AS,ADSGn.DE,ADYEN.AS,AIR.PA,AIRP.PA,ALVG.DE,ASML.AS,AXAF.PA,BASFn.DE,...,SAN.MC,SAPG.DE,SASY.PA,SCHN.PA,SGEF.PA,SIEGn.DE,STLAM.MI,TTEF.PA,VNAn.DE,VOWG_p.DE
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,67.35,10.344663,67.48,NaN,30.065,63.872086,108.30,50.170,13.810,73.18,...,4.642466,61.740795,72.619232,56.69,36.477679,73.202953,1.859794,39.975,NaN,179.00
2013-01-03,66.54,10.344663,67.18,NaN,30.200,63.433011,108.00,50.340,13.825,72.35,...,4.626329,61.661232,72.430223,56.45,36.636105,73.255365,1.886567,39.855,NaN,178.90
2013-01-04,67.62,10.465124,66.72,NaN,30.800,63.499538,108.20,49.755,13.865,72.47,...,4.643200,61.134128,73.285737,56.43,36.754925,73.316513,1.917165,39.820,NaN,178.85
2013-01-07,66.93,10.530375,66.28,NaN,30.750,62.973979,107.70,48.750,13.905,71.64,...,4.637332,60.616969,72.997250,56.10,36.403416,73.045715,1.919078,39.155,NaN,175.95
2013-01-08,65.90,10.460105,66.36,NaN,30.925,62.767747,107.05,48.220,13.575,71.14,...,4.650535,60.318608,73.365320,56.12,36.304400,72.556530,1.895173,39.385,NaN,174.00


In [28]:
# summarize the dataset
EuroStoxx50.describe()

,ABI.BR,AD.AS,ADSGn.DE,ADYEN.AS,AIR.PA,AIRP.PA,ALVG.DE,ASML.AS,AXAF.PA,BASFn.DE,...,SAN.MC,SAPG.DE,SASY.PA,SCHN.PA,SGEF.PA,SIEGn.DE,STLAM.MI,TTEF.PA,VNAn.DE,VOWG_p.DE
count,2595.000000,2595.000000,2568.000000,1204.000000,2595.000000,2595.000000,2568.000000,2595.000000,2595.000000,2568.000000,...,2593.000000,2568.000000,2595.000000,2595.000000,2595.000000,2568.000000,2574.000000,2595.000000,2435.000000,2568.000000
mean,78.466609,20.222655,168.117960,1313.152865,79.464584,94.856556,169.402371,228.457549,21.497944,69.981291,...,4.117864,87.953312,80.714696,83.908960,72.686856,99.203473,8.653848,44.765356,34.876696,159.223306
std,22.017982,4.959865,78.600789,634.775495,27.851830,25.424532,34.043809,192.493032,3.408427,13.209816,...,1.239889,23.157071,7.999882,31.224451,18.878874,20.444696,4.366842,6.105825,11.245108,31.092819
min,30.970000,10.344663,53.890000,411.000000,30.065000,59.481338,101.750000,47.200000,12.494000,38.850000,...,1.460862,50.621882,62.880297,45.930000,34.155742,54.880885,1.859794,21.795000,15.654411,87.200000
25%,56.970000,17.043070,85.000000,680.150000,53.915000,72.103553,138.450000,85.510000,18.907000,62.605000,...,3.176468,65.689103,74.653565,62.020000,56.315000,84.843162,4.683985,40.917500,26.520376,137.600000
50%,76.300000,20.175000,172.775000,1366.000000,72.380000,87.479356,174.655000,148.500000,22.165000,69.815000,...,4.009865,90.157344,79.801572,69.820000,77.220000,94.914723,7.764398,45.280000,33.500000,153.630000
75%,98.465000,23.472500,235.137500,1789.000000,105.010000,120.072733,198.460000,317.450000,23.967500,80.377500,...,5.202168,105.045614,86.635739,102.875000,88.310000,108.752134,12.067848,48.572500,43.469417,179.927500
max,123.250000,31.090000,336.250000,2766.000000,139.000000,150.600015,232.000000,770.500000,28.830000,97.670000,...,6.771846,141.482691,105.665956,177.820000,109.540000,157.960000,19.140000,60.710000,58.327952,255.200000


In [29]:
# drop the columns that only contain NaN
EuroStoxx50 = EuroStoxx50.dropna(axis=1, how='all')
# for those who have NaN, fill them with the previous value
EuroStoxx50 = EuroStoxx50.ffill()

In [30]:
# create the log-returns dataframe
returns = np.log(EuroStoxx50/EuroStoxx50.shift(1))
returns = returns.dropna(axis=0, how='all')

returns.head()

,ABI.BR,AD.AS,ADSGn.DE,ADYEN.AS,AIR.PA,AIRP.PA,ALVG.DE,ASML.AS,AXAF.PA,BASFn.DE,...,SAN.MC,SAPG.DE,SASY.PA,SCHN.PA,SGEF.PA,SIEGn.DE,STLAM.MI,TTEF.PA,VNAn.DE,VOWG_p.DE
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-03,-0.012100,0.000000,-0.004456,NaN,0.004480,-0.006898,-0.002774,0.003383,0.001086,-0.011407,...,-0.003482,-0.001289,-0.002606,-0.004243,0.004334,0.000716,0.014293,-0.003006,NaN,-0.000559
2013-01-04,0.016101,0.011578,-0.006871,NaN,0.019673,0.001048,0.001850,-0.011689,0.002889,0.001657,...,0.003640,-0.008585,0.011742,-0.000354,0.003238,0.000834,0.016089,-0.000879,NaN,-0.000280
2013-01-07,-0.010257,0.006216,-0.006617,NaN,-0.001625,-0.008311,-0.004632,-0.020406,0.002881,-0.011519,...,-0.001265,-0.008495,-0.003944,-0.005865,-0.009610,-0.003700,0.000997,-0.016841,NaN,-0.016348
2013-01-08,-0.015509,-0.006695,0.001206,NaN,0.005675,-0.003280,-0.006054,-0.010931,-0.024019,-0.007004,...,0.002843,-0.004934,0.005030,0.000356,-0.002724,-0.006719,-0.012535,0.005857,NaN,-0.011145
2013-01-09,-0.018996,-0.005292,0.013174,NaN,0.038070,-0.002653,-0.018384,0.012571,0.005510,0.003788,...,0.009575,0.006082,0.001490,0.001425,0.011794,0.002045,0.018991,0.003675,NaN,-0.013889


# Point 0: Variance-Covariance method for VaR and ES in a linear portfolio

On the 20th of February 2020 we have an equally weighted portfolio made up of the following equities

- Adidas
- Allianz
- Munich Re
- L'Oréal

We compute the daily VaR and ES with a 5y estimation using a t-student distribution with 4 degrees
of freedom ($\nu$).
The notional of the portfolio is 15 million €. We take a significance level of $\alpha = 0.99\%$.

Wherever we have missing data due to differing trading days for each stock we substitute the previous
day's value.

## Select the data 

In [31]:
# create a dataframe with the relevant time series
df_0 = returns[['ADSGn.DE', 'ALVG.DE', 'MUVGn.DE', 'OREP.PA']]
# set the date to 20th February 2020
valuation_date_0 = datetime(2020, 2, 20)
# only use data prior to the valuation date
df_0 = df_0[df_0.index < valuation_date_0]
# only use the last 5 years
df_0 = df_0[df_0.index > valuation_date_0 - pd.DateOffset(years=5)]

## Set the parameters

In [32]:
# set nu and alpha
nu_0 = 4
alpha_0 = 0.99
notional_0 = 15 * 10**6

# estimate the mean vector
mean_0 = df_0.mean()
# estimate the covariance matrix
Cov_0 = df_0.cov()
# create the weights vector
w_0 = np.array([0.25, 0.25, 0.25, 0.25])

## Daily VaR
We compute the daily VaR using the variance-covariance method. The daily VaR is given by:

$$
VaR_{\alpha} = \underbrace{\bar\mu \cdot \bar\omega}_{\mu} + \underbrace{ \sqrt{\bar\omega^T \Sigma \bar\omega}}_{\sigma} \cdot t^{-1}_{\nu} (\alpha)
$$

Where:

- $\bar\omega$ is the vector of weights of the portfolio
- $\bar\mu$ is the vector of expected returns of the portfolio
- $\Sigma$ is the variance-covariance matrix of the returns of the portfolio
- $t^{-1}_{\nu}(\alpha)$ is the $\alpha$-quantile of the t-student distribution with $\nu$ degrees of freedom

To compute the quantity $t^{-1}_{\nu}(\alpha)$ we use the `t.ppf` function from the `scipy.stats` module.

See [this stackoverflow answer](https://stackoverflow.com/questions/65468026/norm-ppf-vs-norm-cdf-in-pythons-scipy-stats)
and [this documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html#:~:text=ppf(q%2C%20df,cdf%20%E2%80%94%20percentiles).) for more information.


In [33]:
# find the t_alpha quantile
t_alpha = t.ppf(alpha_0, nu_0)

# compute the VaR
VaR = mean_0 @ w_0 + np.sqrt(w_0 @ Cov_0 @ w_0) * t_alpha

print(f'The daily VaR at 99% confidence level is {VaR:.2%}')
print(f'The daily VaR at 99% confidence level is {VaR * notional_0:.2f} EUR')

The daily VaR at 99% confidence level is 3.86%
The daily VaR at 99% confidence level is 579028.03 EUR


## Daily ES

We compute the daily ES using the variance-covariance method. The daily ES is given by:

$$
ES_{\alpha} = \bar\mu \cdot \bar\omega +
    \sqrt{\bar\omega^T \Sigma \bar\omega} \cdot
    \underbrace{
        \frac{\nu + ( t^{-1}_{\nu}(\alpha) )^2}{\nu - 1} \cdot \frac{ \phi_{\nu} (t^{-1}_{\nu}(\alpha)) }{1 - \alpha}
    }_{ES_{\alpha}^{std}}
$$

Where:

- $\bar\omega$ is the vector of weights of the portfolio
- $\bar\mu$ is the vector of expected returns of the portfolio
- $\Sigma$ is the variance-covariance matrix of the returns of the portfolio
- $t^{-1}_{\nu}(\alpha)$ is the $\alpha$-quantile of the t-student distribution with $\nu$ degrees of freedom
- $\phi_{\nu} (\cdot)$ is the density function of the t-student distribution with $\nu$ degrees of freedom


In [34]:
# compute the ES for the standard t-distribution
ES_std = (nu_0 + t_alpha**2) / (nu_0 - 1) * (t.pdf(t_alpha, nu_0) / (1 - alpha_0))

# compute the ES for the portfolio
ES = mean_0 @ w_0 + np.sqrt(w_0 @ Cov_0 @ w_0) * ES_std

print(f'The daily ES at 99% confidence level is {ES * notional_0:.2f} EUR')

The daily ES at 99% confidence level is 803402.00 EUR


# Point 1: Historical simulation, bootstrap and PCA for VaR and ES in a linear portfolio

On the 20th of March 2019 we must compute the following quantitities with $\alpha = 0.95\%$:

- Portfolio 1: Total (25K shares), AXA (20K shares), Sanofi (20K shares), Volkswagen (10K shares).
    We compute the daily VaR and ES with a Historical Simulation and Bootstrap method (with 200 simulations) and a 5 years estimation.
- Portfolio 2: Adidas, Airbus, BBVA, BMW and Deutsche Telekom (all equally weighted).
    We compute the daily VaR and ES with a 5 year estimation using a Weighted Historical Simulation with $\lambda = 0.95$.
- Portfolio 3: An equally weighted portfolio with shares of the first 18 companies.
    We compute the 10 days VaR and ES with a 5 year estimation using a Gaussian parametric PCA approach using the first n principanl components (with n = 1, 2, 3, 4, 5).

For each portfolio we also check the Plausibility Check.


## Data setup
We set the parameters for the various models and select the data to use

In [35]:
# set the parameters
alpha_1 = 0.95
lmd_1 = 0.94 # lambda is a reserved keyword
# set the valuation date to 20th March 2019
valuation_date_1 = datetime(2019, 3, 20)

# select only the relevant returns
df_1 = returns[returns.index <= valuation_date_1]
# only use the last 5 years (excluding the valuation date)
df_1 = df_1[df_1.index > valuation_date_1 - pd.DateOffset(years=5)]

## Point 1.1 Portfolio 1
First of all we set up the weights we will use to compute the various quantities.
Recall that portfolio 1 has the following quantities:
- Total (25K shares)
- AXA (20K shares)
- Sanofi (20K shares)
- Volkswagen (10K shares)

In [36]:
# select the relevant indexes
df_1_1 = df_1[['TTEF.PA', 'AXAF.PA', 'SASY.PA', 'VOWG_p.DE']]
# compute the value at valuation date
val_Total = 25_000 * EuroStoxx50.loc[valuation_date_1]['TTEF.PA']
val_AXA = 20_000 * EuroStoxx50.loc[valuation_date_1]['AXAF.PA']
val_Sanofi = 20_000 * EuroStoxx50.loc[valuation_date_1]['SASY.PA']
val_VW = 10_000 * EuroStoxx50.loc[valuation_date_1]['VOWG_p.DE']
V_t_1_1 = val_Total + val_AXA + val_Sanofi + val_VW
# compute the weights
w_1_1 = np.array([val_Total, val_AXA, val_Sanofi, val_VW]) / V_t_1_1

### Historical Simulation approach

In order to apply the Historical Simulation approach we need to compute the losses for each day as:

$$
L_t = - V_t \cdot \omega \cdot x_t
$$

then we sort the losses in descending order (the value of index 0 is the highest loss) and we take
the $\alpha$-quantile of the historical losses as $q_{\alpha, HS} = \lfloor (1 - \alpha) \cdot N \rfloor$, with 
$N$ the number of observations.

Then we simply take:

$$

VaR_{\alpha} =  L^{(q_{\alpha, HS}, N)} \\

ES_{\alpha} = \frac{1}{q_{\alpha, HS}} \sum_{i=1}^{q_{\alpha, HS}} L^{(i, N)}

$$

In [37]:
ES_HS, VaR_HS = HSMeasurements(df_1_1, alpha_1, w_1_1, V_t_1_1, 1)

print(f"""
 >--- Historical Simulation ---<
 alpha = {alpha_1} V_t = {V_t_1_1:.2f} EUR
    -> Daily VaR: {VaR_HS:>10.2f} EUR
    -> Daily ES: {ES_HS:>11.2f} EUR
""")


 >--- Historical Simulation ---<
 alpha = 0.95 V_t = 4731416.99 EUR
    -> Daily VaR:   95569.56 EUR
    -> Daily ES:   143630.83 EUR



### Bootstrap approach

In the bootstrap approach we draw $N$ samples with replacement from the historical losses and we compute the VaR and ES as in the Historical Simulation approach.

In [38]:
sample_1_1 = bootstrapStatistical(200, df_1_1)
ES_BS, VaR_BS = HSMeasurements(sample_1_1, alpha_1, w_1_1, V_t_1_1, 1)

print(f"""
 >--- Bootstrap ---<
 alpha = {alpha_1} V_t = {V_t_1_1:.2f} EUR
    -> Daily VaR: {VaR_BS:>10.2f} EUR
    -> Daily ES: {ES_BS:>11.2f} EUR
""")


 >--- Bootstrap ---<
 alpha = 0.95 V_t = 4731416.99 EUR
    -> Daily VaR:  100454.42 EUR
    -> Daily ES:   174036.41 EUR



### Plausibility Check

In order to check wether the VaR and ES we computed are plausible we can use the following formula:

$$

l_i = VaR_{X_i}(1 - \alpha) \quad u_i = VaR_{X_i}(\alpha) \\

sVaR_{X_i} = \underbrace{V_t \omega_i}_{sens_i} \cdot \frac{ |l_i| + |u_i| }{2}

$$

Where $X_i$ is the $i$-th asset in the portfolio and $\omega_i$ is the weight of the $i$-th asset in the portfolio.
Then the portfolio VaR should be of the following:

$$

VaR_{\alpha}^{ptf} = \sqrt{ \bar{sVaR}^T \Sigma \bar{sVaR} }

$$

Where $\bar{sVaR}$ is the vector of the $sVaR$ of each asset in the portfolio and $\Sigma$ is the variance-covariance matrix of the returns of the portfolio.

In [39]:
VaR_PC_1_1 = plausibilityCheck(df_1_1, w_1_1, alpha_1, V_t_1_1, 1)

print(f"""
 >--- Plausibility Check ---<
    -> Thumb Rule: {VaR_PC_1_1:12.2f} EUR
    -> Against the HS: {VaR_HS:.2f} EUR
    -> Against the BS: {VaR_BS:.2f} EUR
""")


 >--- Plausibility Check ---<
    -> Thumb Rule:     92035.63 EUR
    -> Against the HS: 95569.56 EUR
    -> Against the BS: 100454.42 EUR



## Point 1.2: Portfolio 2

We compute the VaR and ES with a 5 year estimation using the Weighted Historical Simulation. 
Recall that the Portfolio 2 is composed by: 
- Adidas
- Airbus
- BBVA
- BMW
- Deutsche Telekom
all equally weighted. In order to have easier calculations we consider the portfolio weight = 1

In [40]:
df_1_2 = df_1[['ADSGn.DE', 'AIR.PA', 'BBVA.MC', 'BMWG.DE', 'DTEGn.DE']]
V_t_1_2 = 1
w_1_2 = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

In [41]:
# compute the ES and VaR with the Weighted Historical Simulation
ES_WHS, VaR_WHS = WHSMeasurements(df_1_2, alpha_1, lmd_1, w_1_2, V_t_1_2, 1)

print(f"""
 >--- Weighted Historical Simulation ---<
 alpha = {alpha_1} lambda = {lmd_1}
    -> Daily VaR: {VaR_WHS:>10.2%}
    -> Daily ES: {ES_WHS:>11.2%}
""")


 >--- Weighted Historical Simulation ---<
 alpha = 0.95 lambda = 0.94
    -> Daily VaR:      1.60%
    -> Daily ES:       2.16%



### Plausibility Check

In [42]:
VaR = plausibilityCheck(df_1_2, w_1_2, alpha_1, V_t_1_2, 1)

print(f"""
    >--- Plausibility Check ---<
    -> Thumb Rule: {VaR:.2%} EUR
""")


    >--- Plausibility Check ---<
    -> Thumb Rule: 1.92% EUR



## Point 1.3: Portfolio 3

We compute the 10 days VaR and ES with a 5 year estimation using a Gaussian parametric PCA approach using the first n principanl components (with n = 1, 2, 3, 4, 5).

In [43]:
# take the first 18 companies without any NaN values
df_1_3 = df_1.dropna(axis=1, how='any').iloc[:, :18]
V_t_1_3 = 1
w_1_3 = np.ones(18) / 18

### PCA approach

We compute the PCA on the returns for each n = 1, 2, 3, 4, 5 and use the first n principal components to compute the VaR and ES.

In [44]:
# compute the ES and VaR with PCA for each n
yearly_mu_1_3 = 256 * df_1_3.mean()
yearly_C_1_3 = 256 * df_1_3.cov()

for i in range(1,6):
    ES_PCA, VaR_PCA = PrincCompAnalysis(yearly_C_1_3, yearly_mu_1_3, w_1_3, 10/256, alpha_1, i, V_t_1_3)
    print(f"""
 >--- PCA with n = {i} ---<
 alpha = {alpha_1}
    -> Daily VaR: {VaR_PCA:>10.2%}
    -> Daily ES: {ES_PCA:>11.2%}
    """)

ValueError: too many values to unpack (expected 2)

In [ ]:
# compute the thumb rule
VaR = plausibilityCheck(df_1_3, w_1_3, alpha_1, V_t_1_3, 10)

print(f"Plausibility Check: {VaR:.2%}")

Plausibility Check: 5.46%


Point 2

In [ ]:
# get the value on the 16 Jan 2016 of BMWG.DE
val_BMWG = EuroStoxx50.loc[datetime(2017, 1, 16)]['BMWG.DE']
print(f'The value of BMWG.DE on 16 Jan 2017 is {val_BMWG:.2f} EUR')

The value of BMWG.DE on 16 Jan 2017 is 86.53 EUR


## POINT 3 - Pricing in case of counterparty risk

In [51]:
#Data
L = 0.99
volatility = 0.2 
settlement_date_3 = datetime(2008, 2, 19)
observation_dates = [settlement_date_3 + relativedelta(years=i) for i in range(8)]

Notional = 30*(10**6)

business_days = pd.bdate_range(start=min(observation_dates), end=max(observation_dates))

# we want to shuft all the non business days at the next business day
for i in range(len(observation_dates)):
    if observation_dates[i] not in business_days:
        next_business_day = business_days[business_days > observation_dates[i]].min()
        observation_dates[i] = next_business_day

The observation dates are: [datetime.datetime(2008, 2, 19, 0, 0), datetime.datetime(2009, 2, 19, 0, 0), datetime.datetime(2010, 2, 19, 0, 0), Timestamp('2011-02-21 00:00:00'), Timestamp('2012-02-20 00:00:00'), datetime.datetime(2013, 2, 19, 0, 0), datetime.datetime(2014, 2, 19, 0, 0), datetime.datetime(2015, 2, 19, 0, 0)]


In [52]:
#Gaussian simulation
mean = 0
simul = 1000
samples = np.random.normal(mean, np.sqrt(volatility), simul)
S = np.zeros(8)
S[0] = 1
Price = np.zeros(simul)
for i in range (simul):
    coupons = np.zeros(7)
    cumulative_sum = np.zeros(8)
    for k in range(7): 
        S[k+1] = S [k]*np.exp(-volatility**2/2+volatility*samples[i])
        coupons[k] = max(0, L*S[k+1]-S[k])
    Price[i] = sum(coupons)
Final_Price = np.mean(Price)
Final_Price

0.23221099287770783